In [16]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [17]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-04 21:44:40.527 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 3367.07it/s]


In [18]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [19]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [20]:
model = rnn_models.BaseRNN(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    horizon=20,
)

In [21]:
yhat = model(x)

In [22]:
loss_fn = torch.nn.CrossEntropyLoss()

In [23]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [24]:
settings = TrainerSettings(
    epochs=3, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 3
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [25]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [26]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 3

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_orig")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=64,
            num_layers=1,
            output_size=20,
            dropout=0.1,
        )

        model = GRUmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
2025-10-04 21:44:42.083 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251004-214442
2025-10-04 21:44:42.084 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Repeat 1/3


100%|██████████| 81/81 [00:00<00:00, 82.97it/s]
2025-10-04 21:44:43.181 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.9302 test 2.7585 metric ['0.1531']
100%|██████████| 81/81 [00:00<00:00, 88.02it/s]
2025-10-04 21:44:44.220 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.3488 test 2.2310 metric ['0.2062']
100%|██████████| 81/81 [00:00<00:00, 90.84it/s]
2025-10-04 21:44:45.233 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.0676 test 1.9647 metric ['0.3109']
100%|██████████| 3/3 [00:03<00:00,  1.05s/it]
2025-10-04 21:44:45.269 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251004-214445
2025-10-04 21:44:45.270 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Repeat 2/3


100%|██████████| 81/81 [00:00<00:00, 83.20it/s]
2025-10-04 21:44:46.358 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.8320 test 2.4567 metric ['0.1266']
100%|██████████| 81/81 [00:00<00:00, 88.83it/s]
2025-10-04 21:44:47.386 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.3140 test 2.2604 metric ['0.2031']
100%|██████████| 81/81 [00:00<00:00, 88.17it/s]
2025-10-04 21:44:48.421 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.1295 test 2.0671 metric ['0.2609']
100%|██████████| 3/3 [00:03<00:00,  1.05s/it]
2025-10-04 21:44:48.457 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251004-214448
2025-10-04 21:44:48.458 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Repeat 3/3


100%|██████████| 81/81 [00:00<00:00, 82.39it/s]
2025-10-04 21:44:49.562 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.8616 test 2.5093 metric ['0.1297']
100%|██████████| 81/81 [00:00<00:00, 88.65it/s]
2025-10-04 21:44:50.597 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.2781 test 2.1792 metric ['0.2156']
100%|██████████| 81/81 [00:00<00:00, 85.38it/s]
2025-10-04 21:44:51.669 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.0525 test 1.9659 metric ['0.3094']
100%|██████████| 3/3 [00:03<00:00,  1.07s/it]


In [27]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""